Install Dependencies:


1.   Java 8
2.   Apache Spark with hadoop and
3.   Findspark (used to locate the spark in the system)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


Set Environment Variables:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
#sc = SparkContext()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We are going to download some datasets called MovieLens
https://grouplens.org/datasets/movielens/

Let's use ML-25M dataset
- ratings.csv
- movies.csv


In [ ]:
sc = SparkContext.getOrCreate();
from pyspark.sql import SparkSession
spark = SparkSession(sc)
movies = spark.read.csv('movies.csv',inferSchema=True, header=True)

In [ ]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

Can you find how many movies are there in movies.csv?  

In [ ]:
movies.count()

62423

In [ ]:
from pyspark.sql.functions import max
movies.select([max("movieId")]).show()

+------------+
|max(movieId)|
+------------+
|      209171|
+------------+



In [ ]:
movies.select("movieId").rdd.max()

Row(movieId=209171)

In [ ]:
movies.select("movieId").rdd.max()[0]

209171

In [ ]:
rdd = movies.select("genres").rdd

In [ ]:
rdd2 = rdd.map(lambda x: x[0])

In [ ]:
rdd3 = rdd.map(lambda x: len(x[0].split('|')))

In [ ]:
from pyspark.sql import Row
def customColumn(row):
  rD = row.asDict()
  rD["countGenres"] = len(row["genres"].split('|'))
  new_row = Row(**rD)
  return new_row

In [ ]:
moviesrdd = movies.rdd
outputDF = moviesrdd.map(customColumn).toDF()
outputDF.show()

+-------+--------------------+--------------------+-----------+
|movieId|               title|              genres|countGenres|
+-------+--------------------+--------------------+-----------+
|      1|    Toy Story (1995)|Adventure|Animati...|          5|
|      2|      Jumanji (1995)|Adventure|Childre...|          3|
|      3|Grumpier Old Men ...|      Comedy|Romance|          2|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|          3|
|      5|Father of the Bri...|              Comedy|          1|
|      6|         Heat (1995)|Action|Crime|Thri...|          3|
|      7|      Sabrina (1995)|      Comedy|Romance|          2|
|      8| Tom and Huck (1995)|  Adventure|Children|          2|
|      9| Sudden Death (1995)|              Action|          1|
|     10|    GoldenEye (1995)|Action|Adventure|...|          3|
|     11|American Presiden...|Comedy|Drama|Romance|          3|
|     12|Dracula: Dead and...|       Comedy|Horror|          2|
|     13|        Balto (1995)|Adventure|

Read ratings

In [ ]:
ratings = spark.read.csv('ratings.csv',inferSchema=True, header=True)

NameError: ignored

In [ ]:
ratings.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
+------+-------+------+----------+
only showing top 10 rows



In [ ]:
ratings = ratings.drop(ratings.timestamp)

In [ ]:
ratings.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
+------+-------+------+
only showing top 10 rows



In [ ]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [ ]:
ratings = ratings.withColumn("movieId",ratings.movieId.cast('integer'))
ratings = ratings.withColumn("userId",ratings.userId.cast('long'))

ratings.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
+------+-------+------+
only showing top 10 rows



Can you check any duplicated reviews?
same userID, same movieID....

In [ ]:
ratings.printSchema()

root
 |-- userId: long (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [ ]:
ratings.groupby(['userId', 'movieId']).count().show()

+------+-------+-----+
|userId|movieId|count|
+------+-------+-----+
|     2|   3479|    1|
|     2|   4874|    1|
|     3|  45447|    1|
|     4| 102125|    1|
|     5|     95|    1|
|     8|    110|    1|
|    12|    337|    1|
|    12|    356|    1|
|    12|   1411|    1|
|    12|   1642|    1|
|    12|   2321|    1|
|    12|   3712|    1|
|    13|   5459|    1|
|    15|   1198|    1|
|    17|   6377|    1|
|    18|    588|    1|
|    19|  86279|    1|
|    23|    260|    1|
|    23|   1726|    1|
|    23|   1962|    1|
+------+-------+-----+
only showing top 20 rows



Now we need to check if 'count' is above 1

In [ ]:
ratings.groupby(['userId', 'movieId']).count().where('count > 1') \
  .sort('count', ascending=False).show()

+------+-------+-----+
|userId|movieId|count|
+------+-------+-----+
+------+-------+-----+



 Build your movie recommender system

---





*   Popularity based model



If you want to build a RS based on popularity what would you do with the two dataset?


any thoughts?





























Yes! Let's count the number of reviews...

In [ ]:
ratingsCnt = ratings.groupBy("movieId").count()

In [ ]:
ratingsCnt.show()

+-------+-----+
|movieId|count|
+-------+-----+
|     29|  293|
|   3091|   42|
|  60756|  119|
|  71530|  122|
|  72011|  232|
| 106002|  141|
| 106100|  220|
|    474|  659|
|   2927|   32|
|   2529|  446|
|  60408|   12|
|     26|   93|
|   5385|   26|
|  96829|  103|
|   1950|  103|
|  51418|    8|
|   2453|   43|
|   4823|  148|
|  71429|   19|
|  72485|    1|
+-------+-----+
only showing top 20 rows



Now, how do I get the most popular movie?   Like top 10 popular movies?

In [ ]:
ratingsCnt.sort('count', ascending=False).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356| 2875|
|    318| 2867|
|    296| 2814|
|    593| 2641|
|   2571| 2514|
|    260| 2411|
|    480| 2267|
|    527| 2103|
|   2959| 2071|
|    110| 2059|
|    589| 2034|
|      1| 2019|
|   1196| 2011|
|     50| 1998|
|   1210| 1911|
|   1198| 1903|
|   4993| 1874|
|   2858| 1818|
|    457| 1791|
|    858| 1781|
+-------+-----+
only showing top 20 rows



Are you curious what that movies are????

what is movieID 356?

In [ ]:
movies.where('movieId = 356').show()

+-------+-------------------+--------------------+
|movieId|              title|              genres|
+-------+-------------------+--------------------+
|    356|Forrest Gump (1994)|Comedy|Drama|Roma...|
+-------+-------------------+--------------------+



what about the 2nd popular movie?

In [ ]:
movies.where('movieId = 296').show()

+-------+-------------------+--------------------+
|movieId|              title|              genres|
+-------+-------------------+--------------------+
|    296|Pulp Fiction (1994)|Comedy|Crime|Dram...|
+-------+-------------------+--------------------+



what about the 3rd?

In [ ]:
movies.where('movieId = 318').show()

+-------+--------------------+-----------+
|movieId|               title|     genres|
+-------+--------------------+-----------+
|    318|Shawshank Redempt...|Crime|Drama|
+-------+--------------------+-----------+



Can't see? use truncate function

In [ ]:
movies.where('movieId = 318').show(truncate=False)

+-------+--------------------------------+-----------+
|movieId|title                           |genres     |
+-------+--------------------------------+-----------+
|318    |Shawshank Redemption, The (1994)|Crime|Drama|
+-------+--------------------------------+-----------+



Do we keep doing this with 20, 100 movies?

No! Now is a good time to use join!


https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.join.html

In [ ]:
ratingsCntSorted = ratingsCnt.sort('count', ascending=False)

In [ ]:
ratingsCntSorted.show(10)

+-------+-----+
|movieId|count|
+-------+-----+
|    356| 2875|
|    318| 2867|
|    296| 2814|
|    593| 2641|
|   2571| 2514|
|    260| 2411|
|    480| 2267|
|    527| 2103|
|   2959| 2071|
|    110| 2059|
+-------+-----+
only showing top 10 rows



In [ ]:
ratingsCntSorted.join(movies, on='movieId').show()

+-------+-----+--------------------+--------------------+
|movieId|count|               title|              genres|
+-------+-----+--------------------+--------------------+
|     29|  293|City of Lost Chil...|Adventure|Drama|F...|
|   3091|   42|    Kagemusha (1980)|           Drama|War|
|  60756|  119|Step Brothers (2008)|              Comedy|
|  71530|  122|   Surrogates (2009)|Action|Sci-Fi|Thr...|
|  72011|  232|Up in the Air (2009)|       Drama|Romance|
| 106002|  141| Ender's Game (2013)|Action|Adventure|...|
| 106100|  220|Dallas Buyers Clu...|               Drama|
|    474|  659|In the Line of Fi...|     Action|Thriller|
|   2927|   32|Brief Encounter (...|       Drama|Romance|
|   2529|  446|Planet of the Ape...| Action|Drama|Sci-Fi|
|  60408|   12|Welcome to the St...|              Comedy|
|     26|   93|      Othello (1995)|               Drama|
|   5385|   26|Last Waltz, The (...|         Documentary|
|  96829|  103|Hunt, The (Jagten...|               Drama|
|   1950|  103

what happened to the order of ratingCntSorted?

This is because of the shuffling under the hood

In [ ]:
ratingsCntSorted.join(movies, on='movieId').orderBy('count', ascending=False).show()

+-------+-----+--------------------+--------------------+
|movieId|count|               title|              genres|
+-------+-----+--------------------+--------------------+
|    356| 2875| Forrest Gump (1994)|Comedy|Drama|Roma...|
|    318| 2867|Shawshank Redempt...|         Crime|Drama|
|    296| 2814| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    593| 2641|Silence of the La...|Crime|Horror|Thri...|
|   2571| 2514|  Matrix, The (1999)|Action|Sci-Fi|Thr...|
|    260| 2411|Star Wars: Episod...|Action|Adventure|...|
|    480| 2267|Jurassic Park (1993)|Action|Adventure|...|
|    527| 2103|Schindler's List ...|           Drama|War|
|   2959| 2071|   Fight Club (1999)|Action|Crime|Dram...|
|    110| 2059|   Braveheart (1995)|    Action|Drama|War|
|    589| 2034|Terminator 2: Jud...|       Action|Sci-Fi|
|      1| 2019|    Toy Story (1995)|Adventure|Animati...|
|   1196| 2011|Star Wars: Episod...|Action|Adventure|...|
|     50| 1998|Usual Suspects, T...|Crime|Mystery|Thr...|
|   1210| 1911

Now, let's extend our previous 'popularity'-based model...

Anything we forgot?

Yes, we didn't use ratings!

Can we get average rating scores for each movies?

In [ ]:
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F

Pyspark sql supports lot of functions :)

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

see aggregate functions !!!

In [ ]:
ratings.groupBy("movieId").agg(F.mean('rating')).show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|     29|3.9108723135271806|
|   3091| 4.049411764705883|
|  60756|3.3919098143236073|
|  71530| 3.277027027027027|
|  72011|3.7143366619115548|
| 106002| 3.391304347826087|
| 106100|3.8762135922330097|
|    474|3.7230081906180192|
|   2927|4.0618181818181816|
|   2529|3.6230748446365846|
|  60408|3.7848837209302326|
|     26|3.6283185840707963|
|   5385| 4.062730627306273|
|  96829| 4.010617760617761|
|   1950| 4.054744525547445|
|  51418| 3.109090909090909|
|   2453|3.0966257668711656|
|   4823|3.3626168224299064|
|  71429|3.3524590163934427|
|  72485|           3.71875|
+-------+------------------+
only showing top 20 rows



what about I want to aggregate more than 1... (mean and count)

In [ ]:
ratings.groupBy("movieId").agg(F.mean('rating'), F.count('rating')).show()

+-------+------------------+-------------+
|movieId|       avg(rating)|count(rating)|
+-------+------------------+-------------+
|     29|3.9108723135271806|         2373|
|   3091| 4.049411764705883|          425|
|  60756|3.3919098143236073|          754|
|  71530| 3.277027027027027|          592|
|  72011|3.7143366619115548|         1402|
| 106002| 3.391304347826087|          598|
| 106100|3.8762135922330097|         1030|
|    474|3.7230081906180192|         5372|
|   2927|4.0618181818181816|          275|
|   2529|3.6230748446365846|         3701|
|  60408|3.7848837209302326|           86|
|     26|3.6283185840707963|          791|
|   5385| 4.062730627306273|          271|
|  96829| 4.010617760617761|          518|
|   1950| 4.054744525547445|          959|
|  51418| 3.109090909090909|           55|
|   2453|3.0966257668711656|          326|
|   4823|3.3626168224299064|         1070|
|  71429|3.3524590163934427|          122|
|  72485|           3.71875|           16|
+-------+--

Do you want to apply a function on both column and create another column?

Let's use UDF! (feel free to search )

In [ ]:
ratingsAgg = ratings.groupBy("movieId").agg(F.mean('rating'), F.count('rating'))

In [ ]:
from pyspark.sql.functions import udf


rs = udf(lambda x : x )     # you try here....!
rsScores = ratingsAgg.withColumn('rsScore', rs(ratingsAgg['avg(rating)']))
rsScores.show()

+-------+------------------+-------------+------------------+
|movieId|       avg(rating)|count(rating)|           rsScore|
+-------+------------------+-------------+------------------+
|     29|3.9108723135271806|         2373|3.9108723135271806|
|   3091| 4.049411764705883|          425| 4.049411764705883|
|  60756|3.3919098143236073|          754|3.3919098143236073|
|  71530| 3.277027027027027|          592| 3.277027027027027|
|  72011|3.7143366619115548|         1402|3.7143366619115548|
| 106002| 3.391304347826087|          598| 3.391304347826087|
| 106100|3.8762135922330097|         1030|3.8762135922330097|
|    474|3.7230081906180192|         5372|3.7230081906180192|
|   2927|4.0618181818181816|          275|4.0618181818181816|
|   2529|3.6230748446365846|         3701|3.6230748446365846|
|  60408|3.7848837209302326|           86|3.7848837209302326|
|     26|3.6283185840707963|          791|3.6283185840707963|
|   5385| 4.062730627306273|          271| 4.062730627306273|
|  96829

In [ ]:
avg functions